In [1]:
#!unzip -qq crystal_ball_data.zip

In [2]:
!pip install transformers
!pip install torch

In [3]:
import torch
from __future__ import print_function
import re, glob, sys, time, os, random
from time import gmtime, strftime
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
import pandas as pd
import warnings
from sklearn.model_selection import cross_val_predict, cross_val_score
from statistics import mean
from datetime import datetime
from time import time
import logging
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
warnings.filterwarnings("ignore", category=UserWarning)
import pprint
from random import shuffle
from typing import List
from typing import Tuple, List, Callable, Optional
from torch import FloatTensor, LongTensor
from torch import nn
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from typing import List
import random
import sklearn
from math import ceil

pipeline = Pipeline([
	('tfidf', TfidfVectorizer(analyzer='word')),
	('clf', LinearSVC())
])


parameters = {
	'tfidf__ngram_range': [(1,2),(1,1),(1,3),(1,4),(2,2),(2,3),(2,4),(3,3),(3,4),(4,4)],
	#'tfidf__analyzer': ('word', 'char'),
	'tfidf__lowercase': (True, False),
	#'tfidf__max_df': (0.01, 1.0), # ignore words that occur as more than 1% of corpus
	'tfidf__min_df': (1, 2, 3), # we need to see a word at least (once, twice, thrice)
	'tfidf__use_idf': (False, True),
	#'tfidf__sublinear_tf': (False, True),
	'tfidf__binary': (False, True),
	'tfidf__norm': (None, 'l1', 'l2'),
	#'tfidf__max_features': (None, 2000, 5000),
	'tfidf__stop_words': (None, 'english'),

	#'tfidfchar_ngram_range': ((1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(2,2),(2,3),(2,4),(2,5),(2,6),(3,3),(3,4),(3,5),(3,6),(4,4),(4,5),(4,6),(5,5),(5,6),(1,7),(2,7),(3,7),(4,7),(5,7),(6,7),(7,7)),
	
	
	'clf__C':(0.1, 1, 5)
}


def balance(Xtrain,Ytrain):
	#v = Ytrain.count('violation')
   # nv = Ytrain.count('non-violation')
	#print(v, nv)
	v = [i for i,val in enumerate(Ytrain) if val=='violation']
	nv = [i for i,val in enumerate(Ytrain) if val=='non-violation']
	if len(nv) < len(v):
		v = v[:len(nv)]
		Xtrain = [Xtrain[j] for j in v] + [Xtrain[i] for i in nv]
		Ytrain = [Ytrain[j] for j in v] + [Ytrain[i] for i in nv]
	if len(nv) > len(v):
		nv = nv[:len(v)]
		Xtrain = [Xtrain[j] for j in v] + [Xtrain[i] for i in nv]
		Ytrain = [Ytrain[j] for j in v] + [Ytrain[i] for i in nv]
	
	#print(Ytrain.count('violation'),Ytrain.count('non-violation'))
	#print('LEN', len(Xtrain), len(Ytrain))
	return Xtrain, Ytrain
	
	

def extract_text(starts, ends, cases, violation):
	facts = []
	D = []
	years = []
	for case in cases:
		contline = ''
		year = 0
		with open(case, 'r', encoding="mbcs") as f:
			for line in f:
				#print(line)
				dat = re.search('^([0-9]{1,2}\s\w+\s([0-9]{4}))', line)
				if dat != None:
					year = int(dat.group(2))
					break
			if year>0:
				years.append(year)
				#print(year)
				wr = 0
				for line in f:
					if wr == 0:
						if re.search(starts, line) != None:
							wr = 1
					if wr == 1 and re.search(ends, line) == None:
						contline += line
						contline += '\n'
					elif re.search(ends, line) != None:
						break
				facts.append(contline)
	for i in range(len(facts)):
		D.append((facts[i], violation, years[i])) 
	return D

def extract_parts(article, violation, part, path):
  # Path is the path to the folder that contains all the text files.
	from os import listdir
	from os.path import isfile, join
	cases = [join(path, f) for f in listdir(path)]
	# cases = glob.glob(path)
	#print(cases)

		
	facts = []
	D = []
	years = []
	
	if part == 'relevant_law':
		for case in cases:
			year = 0
			contline = ''
			with open(case, 'r') as f:
				for line in f:
					dat = re.search('^([0-9]{1,2}\s\w+\s([0-9]{4}))', line)
					if dat != None:
						 #date = dat.group(1)
						year = int(dat.group(2))
						break
				if year> 0:
					years.append(year)
					wr = 0
					for line in f:
						if wr == 0:
							if re.search('RELEVANT', line) != None:
								wr = 1
						if wr == 1 and re.search('THE LAW', line) == None and re.search('PROCEEDINGS', line) == None:
							contline += line
							contline += '\n'
						elif re.search('THE LAW', line) != None or re.search('PROCEEDINGS', line) != None:
							break
					facts.append(contline)
		for i in range(len(facts)):
			D.append((facts[i], violation, years[i]))
		
	if part == 'facts':
		starts = 'THE FACTS'
		ends ='THE LAW'
		D = extract_text(starts, ends, cases, violation)
	if part == 'circumstances':
		starts = 'CIRCUMSTANCES'
		ends ='RELEVANT'
		D = extract_text(starts, ends, cases, violation)
	if part == 'procedure':
		starts = 'PROCEDURE'
		ends ='THE FACTS'
		D = extract_text(starts, ends, cases, violation)
	if part == 'procedure+facts':
		starts = 'PROCEDURE'
		ends ='THE LAW'
		D = extract_text(starts, ends, cases, violation)
	return D


def fetch(part, path, article):
  train_v = extract_parts(article, 'violation', part, path+'/train/'+article+'/violation/')
  train_nv = extract_parts(article, 'non-violation', part, path+'/train/'+article+'/non-violation/')
  test_v = extract_parts(article, 'violation', part, path+'/test20/'+article+'/violation/')
  test_nv = extract_parts(article, 'non-violation', part, path+'/test20/'+article+'/non-violation/')
 
  return train_v, train_nv, test_v, test_nv


def get_facts_dataset(articles: List[int], shuffle: bool = False):
  """
  Returns a tuple of (training_data, training_labels, test_data, test labels)
  containing the data from the given articles. The 'data' fields are lists of strings
  that contain the FACTS part of the cases, while the 'labels' fields are also lists of
  strings containing either 'violation' or 'non-violation' for their respective data
  counterparts.
  :param articles: List of integers of article numbers.
  :param shuffle: Randomly shuffles the training set if True.
  """
  path = "crystal_ball_data"
  traind = []
  trainl = []
  testd = []
  testl = []
  for i in articles:
    art = f"Article{i}"
    trv, trnv, tev, tenv = fetch("facts", path, art)
    traind.extend([e[0] for e in trv] + [e[0] for e in trnv])
    trainl.extend([e[1] for e in trv] + [e[1] for e in trnv])

    testd.extend([e[0] for e in tev] + [e[0] for e in tenv])
    testl.extend([e[1] for e in tev] + [e[1] for e in tenv])
  
  if shuffle:
    c = list(zip(traind, trainl))
    random.shuffle(c)

    traind, trainl = zip(*c)
  
  return traind, trainl, testd, testl

In [4]:
pre_trained_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
def train(model: torch.nn.Module, data: torch.Tensor, labels: torch.Tensor, batch_size: int, optimizer, verbose=False):
  """
  Trains the network for one epoch.
  """
  # Set to training mode.
  model.train(True)

  for start_i in range(0, len(data), batch_size):
    x = data[start_i:start_i+batch_size]
    y = labels[start_i:start_i+batch_size]

    y = y.to(torch.float32)
    out = model(x)
    out = out.squeeze(-1)
    #h_n.detach_()
    #h_c.detach_()
    #print(out.shape)
    #print(y.shape)

    #print(y[1].item())
    #print(out[1].item())

    loss = torch.nn.functional.binary_cross_entropy(input=out, target=y)
    #lossfunc = torch.nn.BCELoss()
    #loss = lossfunc(out, y)
    if verbose:
      print(f"loss: {loss.item()}")
    model.zero_grad()
    loss.backward(retain_graph = True)
    optimizer.step()


def test(model: torch.nn.Module, data: torch.Tensor, labels: torch.Tensor, batch_size: int = 32) -> float:
  """
  Calculates classification accuracy on the dataset and returns the result.
  """
  model.train(False)
  accuracy = 0
  weight = 0
  for start_i in range(0, len(data), batch_size):
    x = data[start_i:start_i+batch_size]
    y = labels[start_i:start_i+batch_size]
    weight += len(x)
    with torch.no_grad():
      y_hat = torch.round(model(x)).cpu()
    #print(y_hat)
    accuracy += len(x) * sklearn.metrics.accuracy_score(y, y_hat)
  return accuracy / weight

def save(model, optimizer, epoch):
  torch.save({
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'epoch': epoch
      }, f"/content/drive/My Drive/PR checkpoints/checkpoint_e{epoch}.pt")

In [6]:
# We can use all these article numbers: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18] 
# For training
article_numbers = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18]
training_data, training_labels, test_data, test_labels = get_facts_dataset(article_numbers, shuffle=True)

# Convert it to token IDs.
training_data_token = tokenizer(training_data, return_tensors='pt', padding=True, truncation=True)
# RNN we only need the IDs.
training_data_token["input_ids"]
# Now convert the tokens into vectors.
training_vectors = pre_trained_bert.bert.embeddings(training_data_token["input_ids"])
    
import torch
# Convert labels to 0 and 1
training_labels_binary = torch.tensor([0 if l == "violation" else 1 for l in training_labels], dtype=torch.long)

In [10]:
#(everytime when you swap )
###############################################################################################
import torch

# We can use all these article numbers: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18] 
# For training
article_numbers = [10]
void1, void2, test_data, test_labels = get_facts_dataset(article_numbers, shuffle=True)
# Convert it to token IDs. (everytime when you swap )
test_data_token = tokenizer(test_data, return_tensors='pt', padding=True, truncation=True)
# RNN we only need the IDs.
test_data_token["input_ids"]
# Now convert the tokens into vectors.
test_vectors = pre_trained_bert.bert.embeddings(test_data_token["input_ids"])
# Convert labels to 0 and 1
test_labels_binary = torch.tensor([0 if l == "violation" else 1 for l in test_labels], dtype=torch.long)

###############################################################################################

In [11]:
#change hidden layer size for higher accuracy
hidden_layer_size = 256

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size= 768,
            hidden_size=hidden_layer_size,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(hidden_layer_size, 1)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return torch.nn.functional.sigmoid(out)

rnn = RNN()

In [12]:
#change batch size and learning rate to get high accuracy
batch_size = 32
learning_rate = 0.0001
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate) 


print(f"Size of the training data is: {len(training_vectors)}. Completing an epoch with batch size of {batch_size} will take {ceil(len(training_vectors) / batch_size)} iterations.")
acc = test(rnn, test_vectors, test_labels_binary, batch_size=batch_size)
print(f"| Test accuracy: {acc*100:.3f}%")
for epoch in range(8):
    train(rnn, training_vectors, training_labels_binary, batch_size=batch_size, optimizer=optimizer, verbose=False)
    acc = test(rnn, test_vectors, test_labels_binary, batch_size=batch_size)
    #save(model=rnn, optimizer=optimizer, epoch=epoch+1)
    print(f"=== Epoch {epoch+1} completed. ===")
    print(f"| Test accuracy: {acc*100:.3f}%")

Size of the training data is: 3166. Completing an epoch with batch size of 32 will take 99 iterations.
| Test accuracy: 52.830%
=== Epoch 1 completed. ===
| Test accuracy: 43.396%
=== Epoch 2 completed. ===
| Test accuracy: 43.396%
=== Epoch 3 completed. ===
| Test accuracy: 41.509%
=== Epoch 4 completed. ===
| Test accuracy: 52.830%
=== Epoch 5 completed. ===
| Test accuracy: 52.830%
=== Epoch 6 completed. ===
| Test accuracy: 50.943%
=== Epoch 7 completed. ===
| Test accuracy: 54.717%
=== Epoch 8 completed. ===
| Test accuracy: 54.717%
